In [11]:
from web3 import Web3

In [12]:
# Initialize connection
node_url = 'http://127.0.0.1:9545/'
web3_instance = Web3(Web3.HTTPProvider(node_url))

In [13]:
# Check if connection is established successfully
if web3_instance.is_connected():
    print('connection success')
else:
    raise ValueError('no connection established')

connection success


In [14]:
# Owner
owner_address = web3_instance.eth.accounts[0]

In [15]:
# Contract address
contract_address = '0x9e9044de58f6047e63513c12D8c6A7e53F4C6AFe'

In [16]:
# get ABI of Exchanger
import json
exchanger_contract_path = '/Users/evenmare/pets/donations-solidity-token/build/contracts/DonationExchanger.json'
with open(exchanger_contract_path) as file:
    exchanger_contract_json = json.load(file)
    contract_abi = exchanger_contract_json['abi']

In [17]:
# build exchanger contract scheme
exchanger_contract = web3_instance.eth.contract(address=contract_address, abi=contract_abi)

In [18]:
list(exchanger_contract.functions)  # methods of exchanger

['owner', 'token', 'sell']

In [19]:
exchanger_contract.functions.owner().call()

'0x3Ab188835031d80874eC99856fd01E8E315f50FD'

In [20]:
token_contract_address = exchanger_contract.functions.token().call()
token_contract_address

'0x59d93206ea1Dd18251897F609594175718C1f38D'

In [21]:
# get ABI of Token
import json
token_contract_path = '/Users/evenmare/pets/donations-solidity-token/build/contracts/DonationToken.json'
with open(token_contract_path) as file:
    token_contract_json = json.load(file)
    token_contract_abi = token_contract_json['abi']

In [22]:
# build token contract scheme
token_contract = web3_instance.eth.contract(address=token_contract_address, abi=token_contract_abi)

In [23]:
token_contract.functions  # methods of token

In [24]:
token_contract.functions.totalSupply().call()

300

In [25]:
token_contract.functions.getOwner().call()

'0x9e9044de58f6047e63513c12D8c6A7e53F4C6AFe'

In [33]:
def send_signed_contract_transaction(contract_method):
    private_key = '51a4f2fe64d75da3739c2bfc1e84ac5c1bae5fc29af3840ffe2e5b87fe570079'
    nonce = web3_instance.eth.get_transaction_count(owner_address)
    transaction = contract_method.build_transaction(
        {'nonce': nonce, 'chainId': 1337, 'from': owner_address}
    )
    signed_transaction = web3_instance.eth.account.sign_transaction(
        transaction, private_key=private_key,
    )
    return web3_instance.eth.send_raw_transaction(signed_transaction.rawTransaction)


In [36]:
send_signed_contract_transaction(token_contract.functions.registerMember(
    web3_instance.eth.accounts[1],
))

# token_contract.functions.registerMember(owner_address).transact({'from': owner_address})

In [37]:
token_contract.functions.moderator().call(), owner_address

('0x3Ab188835031d80874eC99856fd01E8E315f50FD',
 '0x3Ab188835031d80874eC99856fd01E8E315f50FD')

In [38]:
token_contract.functions.getMembersRegistry().call()

['0x3Ab188835031d80874eC99856fd01E8E315f50FD',
 '0x50ae8cF59Ebf7D1903f5803aEe533AF761BCE21c']

In [39]:
send_signed_contract_transaction(token_contract.functions.registerDonateNeeds('some shit', 300))

In [40]:
token_contract.functions.donateNeedsNames(owner_address).call()

['some shit']

In [41]:
token_contract.functions.donateNeed(owner_address, 'some shit').call()

300

In [ ]:
# token_contract.functions.registerDonateNeeds('some shit', 300).transact({'from': owner_address})

HexBytes('0xd7c57b0185a8ebd10952ea819957c24669c43766e5630147fba0ee50d63db253')

In [ ]:
# token_contract.functions.donateNeed(owner_address, 'some shit').call()

300

In [ ]:
# token_contract.functions.donateBalance(owner_address).call()

0

In [ ]:
# # web3_instance.eth.send_transaction({
#     'from': web3_instance.eth.accounts[1],
#     'to': contract_address,
#     'value': web3_instance.to_wei('10', 'wei'),
# })

HexBytes('0x82eca1862b30cd55b9df06b7fa278b5e91d4af167d31508a893ba6e47853d8ee')

In [ ]:
# token_contract.functions.balanceOf(owner_address).call(), \
#     token_contract.functions.balanceOf(web3_instance.eth.accounts[1]).call()

(3, 7)

In [ ]:
# token_contract.functions.transfer(owner_address, 3).transact({'from': web3_instance.eth.accounts[1]})

HexBytes('0x1f73735e234e4ce3e478648e05b8ed4c203bdb98af1db6931c94e568379b90a0')